In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [3]:
torch.manual_seed(32)
model = Model()

In [6]:
model

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

## Checking if model parameters on cuda

In [5]:
next(model.parameters()).is_cuda

False

## Sending created model to cuda

In [6]:
gpumodel = model.cuda()

In [7]:
next(gpumodel.parameters()).is_cuda

True

In [8]:
df = pd.read_csv('iris.csv')

In [9]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [10]:
X = df.drop('target',axis=1).values

In [11]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [12]:
y = df['target'].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

In [15]:
X_train = torch.FloatTensor(X_train).cuda()


TypeError: expected Variable[CPUType] (got torch.cuda.FloatTensor)

In [16]:
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

## Do not forget to set pin_memory=True when using DataLoader

In [17]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True,pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False,pin_memory=True)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [20]:
import time
start = time.time()
epochs = 100
losses = []

for i in range(epochs):
    
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred,y_train)
    losses.append(loss)
    
    if i%10 == 0:
        print(f'Epoch: {i} loss {loss.item()}')
        
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


end = time.time() - start
print(f'Total time: {end}')

Epoch: 0 loss 1.1507114171981812
Epoch: 10 loss 0.9377318620681763
Epoch: 20 loss 0.779827356338501
Epoch: 30 loss 0.6099671125411987
Epoch: 40 loss 0.40083539485931396
Epoch: 50 loss 0.25436994433403015
Epoch: 60 loss 0.15052448213100433
Epoch: 70 loss 0.10086147487163544
Epoch: 80 loss 0.08127660304307938
Epoch: 90 loss 0.07230930775403976
Total time: 1.5361850261688232


In [21]:
correct = 0
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f'{i+1:2}.{str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct = correct + 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1.tensor([-2.1245,  4.8074, -0.8623], device='cuda:0')  1
 2.tensor([-1.7976,  5.3110, -1.5445], device='cuda:0')  1
 3.tensor([  6.3540,   0.8432, -10.0531], device='cuda:0')  0
 4.tensor([-3.9114,  4.5970,  1.1698], device='cuda:0')  1
 5.tensor([-7.4707,  3.2028,  5.7866], device='cuda:0')  2
 6.tensor([-10.4965,   1.6465,   9.6306], device='cuda:0')  2
 7.tensor([  6.3199,   0.9912, -10.1520], device='cuda:0')  0
 8.tensor([  7.0468,   0.7053, -10.9127], device='cuda:0')  0
 9.tensor([-7.2053,  3.3485,  5.3575], device='cuda:0')  2
10.tensor([-9.3950,  2.5768,  8.1044], device='cuda:0')  2
11.tensor([-9.8795,  2.3483,  8.7148], device='cuda:0')  2
12.tensor([ 6.2747,  0.6650, -9.7604], device='cuda:0')  0
13.tensor([-9.3131,  2.1888,  8.1954], device='cuda:0')  2
14.tensor([-3.7796,  4.5061,  1.0759], device='cuda:0')  1
15.tensor([-7.8646,  3.0127,  6.2310], device='cuda:0')  2
16.tensor([-1.8859,  5.1583, -1.3341], device='cuda:0')  1
17.tensor([-5.6999,  3.5039,  3.6704], devic